In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import layers

nb_classes = 10
(train_x, train_y), (test_x, test_y) = keras.datasets.cifar10.load_data()
train_x, test_x = train_x / 255.0, test_x / 255.0

train_y = keras.utils.to_categorical(train_y, nb_classes)
test_y = keras.utils.to_categorical(test_y, nb_classes)

170508288/170498071 [==============================] - 2s 0us/step


In [ ]:
class Block(keras.layers.Layer):
  def __init__(self, f1, f2, s):
    super().__init__()
    kernel_initializer = 'he_normal'
    self.strides = s
    
    self.layers = keras.Sequential([
      # implement here
      layers.Conv2D(filters=f1, kernel_size=3, strides=s, padding='same', kernel_initializer=kernel_initializer),
      layers.BatchNormalization(),
      layers.ReLU(),
      layers.Conv2D(filters=f2, kernel_size=3, strides=1, padding='same', kernel_initializer=kernel_initializer),
      layers.BatchNormalization(),
      layers.ReLU(),
    ])
    
  def call(self, inputs):
    x = self.layers(inputs)
    return x

class Network(keras.Model):
  def __init__(self):
    super().__init__()
    self.kernel_initializer = 'he_normal'
  
  def build(self, input_shape): # automatically called when first __call__ invokes.
    self.first_layer = keras.Sequential([
      layers.Conv2D(filters=16, kernel_size=3, strides=1, padding='same', kernel_initializer=self.kernel_initializer),
      layers.ReLU()
    ])
    
    # implement blocks here
    # using for() can make code short
    self.blocks = []
    filters_per_stage = [16, 32, 64]
    strides_of_first_block_per_stage = [1, 2, 2]
    repeats_per_stage = [3, 3, 3]
    for filters, strides, repeats in zip(filters_per_stage, strides_of_first_block_per_stage, repeats_per_stage):
      for _ in range(repeats):
        self.blocks.append(Block(filters, filters, strides))
        strides = 1

    self.last_layer = keras.Sequential([
      layers.GlobalAveragePooling2D(),
      layers.Flatten(),
      layers.Dense(10, kernel_initializer=self.kernel_initializer)
    ])

  def call(self, inputs): # automatically called in __call__ after build().
    x = self.first_layer(inputs)
    
    #implement blocks call here
    for block in self.blocks:
      x = block(x)

    x = self.last_layer(x)

    return x

model = Network()
model.compile(loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              optimizer=keras.optimizers.Adam(0.005),
              metrics=['accuracy'])

epoch = 7
batch_size = 128
model.fit(x=train_x, y=train_y, batch_size=batch_size, epochs=epoch, validation_data=(test_x, test_y))

Epoch 1/7
391/391 [==============================] - 26s 30ms/step - loss: 1.6876 - accuracy: 0.3561 - val_loss: 4.8813 - val_accuracy: 0.2165
Epoch 2/7
391/391 [==============================] - 10s 27ms/step - loss: 1.2930 - accuracy: 0.5229 - val_loss: 1.6043 - val_accuracy: 0.4916
Epoch 3/7
391/391 [==============================] - 11s 27ms/step - loss: 1.0838 - accuracy: 0.6082 - val_loss: 2.0885 - val_accuracy: 0.4263
Epoch 4/7
391/391 [==============================] - 11s 27ms/step - loss: 0.9767 - accuracy: 0.6464 - val_loss: 1.1355 - val_accuracy: 0.5975
Epoch 5/7
391/391 [==============================] - 11s 27ms/step - loss: 0.8852 - accuracy: 0.6846 - val_loss: 1.8817 - val_accuracy: 0.4889
Epoch 6/7
391/391 [==============================] - 11s 27ms/step - loss: 0.8134 - accuracy: 0.7099 - val_loss: 1.6445 - val_accuracy: 0.4921
Epoch 7/7
391/391 [==============================] - 11s 27ms/step - loss: 0.7498 - accuracy: 0.7353 - val_loss: 1.0062 - val_accuracy: 0.6693

In [ ]:
class ResNetBlock(keras.layers.Layer):
  def __init__(self, f1, f2, s):
    super().__init__()
    kernel_initializer = 'he_normal'
    self.strides = s
    
    self.residual = keras.Sequential([
      layers.Conv2D(filters=f1, kernel_size=3, strides=s, padding='same', kernel_initializer=kernel_initializer),
      layers.BatchNormalization(),
      layers.ReLU(),
      layers.Conv2D(filters=f2, kernel_size=3, strides=1, padding='same', kernel_initializer=kernel_initializer),
      layers.BatchNormalization()
    ])
    
    if self.strides == 2:
      self.shortcut_conv = keras.Sequential([
        layers.Conv2D(filters=f2, kernel_size=1, strides=s, kernel_initializer=kernel_initializer, padding='same'),
        layers.BatchNormalization()
      ])

  def call(self, inputs):
    if self.strides == 2:
      shortcut = self.shortcut_conv(inputs)
    else:
      shortcut = inputs
    
    x = self.residual(inputs)
    x = layers.add([x, shortcut]) # Or, x = x + shortcut
    x = layers.ReLU()(x) # Or, x = tf.nn.relu(x)
    
    return x
    
class ResNet(keras.Model):
  def __init__(self):
    super().__init__()
    self.kernel_initializer = 'he_normal'
  
  def build(self, input_shape): # automatically called when first __call__ invokes.
    self.first_layer = keras.Sequential([
      layers.Conv2D(filters=16, kernel_size=3, strides=1, padding='same', kernel_initializer=self.kernel_initializer),
      layers.BatchNormalization(),
      layers.ReLU()
    ])

    self.blocks = []
    filters_per_stage = [16, 32, 64]
    strides_of_first_block_per_stage = [1, 2, 2]
    repeats_per_stage = [3, 3, 3]
    for filters, strides, repeats in zip(filters_per_stage, strides_of_first_block_per_stage, repeats_per_stage):
      for _ in range(repeats):
        self.blocks.append(ResNetBlock(filters, filters, strides))
        strides = 1

    self.last_layer = keras.Sequential([
      layers.GlobalAveragePooling2D(),
      layers.Flatten(),
      layers.Dense(10, kernel_initializer=self.kernel_initializer)
    ])

  def call(self, inputs): # automatically called in __call__ after build().
    x = self.first_layer(inputs)
    for block in self.blocks:
      x = block(x)
    x = self.last_layer(x)

    return x
    
model = ResNet()
model.compile(loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              optimizer=keras.optimizers.Adam(0.001),
              metrics=['accuracy'])

epoch = 7
batch_size = 128
model.fit(x=train_x, y=train_y, batch_size=batch_size, epochs=epoch, validation_data=(test_x, test_y))

Epoch 1/7
391/391 [==============================] - 16s 32ms/step - loss: 1.4145 - accuracy: 0.4815 - val_loss: 1.6486 - val_accuracy: 0.4634
Epoch 2/7
391/391 [==============================] - 13s 32ms/step - loss: 1.0118 - accuracy: 0.6374 - val_loss: 1.0601 - val_accuracy: 0.6234
Epoch 3/7
391/391 [==============================] - 12s 30ms/step - loss: 0.8257 - accuracy: 0.7105 - val_loss: 0.9990 - val_accuracy: 0.6579
Epoch 4/7
391/391 [==============================] - 12s 32ms/step - loss: 0.6979 - accuracy: 0.7548 - val_loss: 1.0032 - val_accuracy: 0.6551
Epoch 5/7
391/391 [==============================] - 13s 32ms/step - loss: 0.6024 - accuracy: 0.7915 - val_loss: 0.9624 - val_accuracy: 0.6816
Epoch 6/7
391/391 [==============================] - 12s 30ms/step - loss: 0.5268 - accuracy: 0.8163 - val_loss: 0.8529 - val_accuracy: 0.7222
Epoch 7/7
391/391 [==============================] - 13s 32ms/step - loss: 0.4602 - accuracy: 0.8390 - val_loss: 0.7941 - val_accuracy: 0.7362